### tf.keras.preprocessing.image.ImageDataGenerator()类

通过该类可以生成一个batch的Tensor图像数据，支持实时的数据增强，训练地时候会无限生成数据直到达到指定的epoches。

常用参数：
* ```rescale```：缩放系数。默认为None,如果为None或0，则不进行任何比例的缩放，否则，将数据乘以提供的值。
* ```rotation_range```：int类型,将图片随机旋转度数的范围。
* ```width_shift_range```：随机左右平移，0~1代表比例，1~代表像素值
* ```height_shift_range```：随机上下平移，0~1代表比例，1~代表像素值
* ```shear_range```：float类型，表示剪切强度（逆时针剪切角，以度为单位）。
* ```zoom_range```：float类型或者是一个范围 [lower,upper]，表示缩放的范围。如果是float类型，[lower, upper] = [1-zoom_range, 1+zoom_range]。

* ```horizontal_flip```：布尔值，对图像水平随机翻转。
* ```vertical_flip```：布尔值，对图像垂直随机翻转。

* ```fill_mode```：{"constant", "nearest", "reflect" or "wrap"}之一，默认值为“nearest”。输入边界之外的点将根据给定模式进行填充。
* ```cval```：浮点型或整数型。fill_mode = "constant"时用于填充边界外点的值。
* ```brightness_range```：元组或两个浮点数的列表。从中选择亮度偏移值的范围。
* ```channel_shift_range```：浮动。随机通道移位的范围。

其他具体参数参见：
https://tensorflow.google.cn/versions/r2.0/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [1]:
# check version

import tensorflow as tf
from tensorflow import keras

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
import numpy as np
import pandas as pd
import os,sys,time

for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

matplotlib 3.1.1
numpy 1.16.0
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0-beta0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


### 通过ImageDataGenerator()类进行数据增强

In [2]:
height = 224
width = 224
channels = 3
batch_size = 64
num_classes = 2

#train data 从猫狗大战数据集里选出200张猫和200张狗的图片作为训练集
train_dir = "./data/cat_vs_dog/train"

#首先给出数据增强的方式，通过参数进行填充配置
train_data_gen_method = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255, #归一化处理
    rotation_range = 40, #图片随机旋转-40~40之间的一个角度
    width_shift_range = 0.2, #随机平移，0~1代表比例，1~代表像素值
    height_shift_range = 0.2,
    shear_range = 0.2, #剪切强度
    zoom_range = 0.2, #缩放强度
    horizontal_flip = True ,#是都随机水平翻转
    fill_mode = 'nearest',   #像素填充方式
)

.flow()方法接收numpy数组和标签为参数,生成经过数据提升或标准化后的batch数据,
 并在一个无限循环中不断的返回batch数据。
 它有如下参数：
* X：样本数据，秩应为4.在黑白图像的情况下channel轴的值为1，在彩色图像情况下值为3
* y：标签
* batch_size：整数，默认32
* shuffle：布尔值，是否随机打乱数据，默认为True
* save_to_dir：None或字符串，该参数能让你将提升后的图片保存起来，用以可视化
* save_prefix：字符串，保存提升后图片时使用的前缀, 仅当设置了save_to_dir时生效
* save_format："png"或"jpeg"之一，指定保存图片的数据格式,默认"jpeg"
* yields:形如(x,y)的tuple,x是代表图像数据的numpy数组.y是代表标签的numpy数组.该迭代器无限循环.
* seed: 整数,随机数种子

In [3]:
# 选择一张图片进行数据增强
image1 = keras.preprocessing.image.load_img('./data/cat_vs_dog/train/cat/cat.1.jpg')
x = keras.preprocessing.image.img_to_array(image1)

# 因为x的是4维的，而一张图片是3维的，所以给它增加一个维度
x = np.expand_dims(x,0)

# 存放增强图像数据的路径
savd_dir = './data/generate_images'
if not os.path.exists(savd_dir):
    os.makedirs(savd_dir)
    
#生成20张
i = 0
for batch in train_data_gen_method.flow(x,batch_size=1,
                                        save_to_dir=savd_dir,
                                        save_prefix='new_Cat',
                                        save_format='jpeg'):
    i+=1
    if i==20:
        break
        
print("Generate 20 images done.")

Generate 20 images done.


<img src="./data/results/1.png" ,width=720, heigth=540>


In [4]:
#从指定目录中读取图片，对每一个图片按照上面的方法进行处理
#这里同样可以使用save_to_dir来保存图片，用法同上，不再演示

train_data_generator = train_data_gen_method.flow_from_directory(
    train_dir,  #对于每一个类,该文件夹都要包含一个子文件夹。
                #子文件夹中任何JPG、PNG和BNP的图片都会被生成器使用。
    target_size = (height,width),#图像将被resize成该尺寸
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = "categorical",#返回2d onehot编码
)

train_num = train_data_generator.samples
print(train_num)

Found 400 images belonging to 2 classes.
400
